# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 8 2022 7:35AM,250253,20,8745856,"Exact-Rx, Inc.",Released
1,Nov 8 2022 7:35AM,250253,20,8745857,"Exact-Rx, Inc.",Released
2,Nov 8 2022 7:35AM,250253,20,8745858,"Exact-Rx, Inc.",Released
3,Nov 8 2022 7:35AM,250253,20,8745859,"Exact-Rx, Inc.",Released
4,Nov 8 2022 7:35AM,250253,20,8745860,"Exact-Rx, Inc.",Released
5,Nov 8 2022 7:35AM,250253,20,8745861,"Exact-Rx, Inc.",Released
6,Nov 8 2022 7:35AM,250253,20,8745862,"Exact-Rx, Inc.",Released
7,Nov 8 2022 7:35AM,250253,20,8745863,"Exact-Rx, Inc.",Released
8,Nov 8 2022 7:35AM,250253,20,8745864,"Exact-Rx, Inc.",Released
9,Nov 8 2022 7:35AM,250253,20,8745865,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,250243,Released,13
31,250244,Released,1
32,250250,Released,1
33,250252,Released,1
34,250253,Released,22


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250243,NaN,NaN,13.0
250244,NaN,NaN,1.0
250250,NaN,NaN,1.0
250252,NaN,NaN,1.0
250253,NaN,NaN,22.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250121,0.0,1.0,0.0
250130,0.0,0.0,1.0
250131,0.0,0.0,1.0
250139,0.0,0.0,1.0
250141,19.0,1.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250121,0,1,0
250130,0,0,1
250131,0,0,1
250139,0,0,1
250141,19,1,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250121,0,1,0
1,250130,0,0,1
2,250131,0,0,1
3,250139,0,0,1
4,250141,19,1,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250121,,1,
1,250130,,,1
2,250131,,,1
3,250139,,,1
4,250141,19,1,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 8 2022 7:35AM,250253,20,"Exact-Rx, Inc."
22,Nov 7 2022 5:12PM,250252,21,"NBTY Global, Inc."
23,Nov 7 2022 4:38PM,250250,10,Methapharm-G
24,Nov 7 2022 4:13PM,250244,16,"Emersa Waterbox, LLC"
25,Nov 7 2022 4:03PM,250243,10,ISDIN Corporation
38,Nov 7 2022 3:57PM,250242,10,"Methapharm, Inc."
56,Nov 7 2022 3:52PM,250241,10,"ClearSpec, LLC"
59,Nov 7 2022 3:50PM,250239,10,"Methapharm, Inc."
61,Nov 7 2022 3:46PM,250236,10,"Methapharm, Inc."
66,Nov 7 2022 3:41PM,250235,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 8 2022 7:35AM,250253,20,"Exact-Rx, Inc.",,,22
1,Nov 7 2022 5:12PM,250252,21,"NBTY Global, Inc.",,,1
2,Nov 7 2022 4:38PM,250250,10,Methapharm-G,,,1
3,Nov 7 2022 4:13PM,250244,16,"Emersa Waterbox, LLC",,,1
4,Nov 7 2022 4:03PM,250243,10,ISDIN Corporation,,,13
5,Nov 7 2022 3:57PM,250242,10,"Methapharm, Inc.",,,18
6,Nov 7 2022 3:52PM,250241,10,"ClearSpec, LLC",,,3
7,Nov 7 2022 3:50PM,250239,10,"Methapharm, Inc.",,,2
8,Nov 7 2022 3:46PM,250236,10,"Methapharm, Inc.",,,5
9,Nov 7 2022 3:41PM,250235,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 7:35AM,250253,20,"Exact-Rx, Inc.",22,,
1,Nov 7 2022 5:12PM,250252,21,"NBTY Global, Inc.",1,,
2,Nov 7 2022 4:38PM,250250,10,Methapharm-G,1,,
3,Nov 7 2022 4:13PM,250244,16,"Emersa Waterbox, LLC",1,,
4,Nov 7 2022 4:03PM,250243,10,ISDIN Corporation,13,,
5,Nov 7 2022 3:57PM,250242,10,"Methapharm, Inc.",18,,
6,Nov 7 2022 3:52PM,250241,10,"ClearSpec, LLC",3,,
7,Nov 7 2022 3:50PM,250239,10,"Methapharm, Inc.",2,,
8,Nov 7 2022 3:46PM,250236,10,"Methapharm, Inc.",5,,
9,Nov 7 2022 3:41PM,250235,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 7:35AM,250253,20,"Exact-Rx, Inc.",22,,
1,Nov 7 2022 5:12PM,250252,21,"NBTY Global, Inc.",1,,
2,Nov 7 2022 4:38PM,250250,10,Methapharm-G,1,,
3,Nov 7 2022 4:13PM,250244,16,"Emersa Waterbox, LLC",1,,
4,Nov 7 2022 4:03PM,250243,10,ISDIN Corporation,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 7:35AM,250253,20,"Exact-Rx, Inc.",22.0,NaN,NaN
1,Nov 7 2022 5:12PM,250252,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Nov 7 2022 4:38PM,250250,10,Methapharm-G,1.0,NaN,NaN
3,Nov 7 2022 4:13PM,250244,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Nov 7 2022 4:03PM,250243,10,ISDIN Corporation,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 7:35AM,250253,20,"Exact-Rx, Inc.",22.0,0.0,0.0
1,Nov 7 2022 5:12PM,250252,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Nov 7 2022 4:38PM,250250,10,Methapharm-G,1.0,0.0,0.0
3,Nov 7 2022 4:13PM,250244,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Nov 7 2022 4:03PM,250243,10,ISDIN Corporation,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3252796,70.0,2.0,19.0
15,500333,106.0,0.0,0.0
16,1000702,4.0,1.0,0.0
18,250131,1.0,0.0,0.0
19,1000727,4.0,0.0,0.0
20,500437,23.0,0.0,0.0
21,1751342,7.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3252796,70.0,2.0,19.0
1,15,500333,106.0,0.0,0.0
2,16,1000702,4.0,1.0,0.0
3,18,250131,1.0,0.0,0.0
4,19,1000727,4.0,0.0,0.0
5,20,500437,23.0,0.0,0.0
6,21,1751342,7.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,70.0,2.0,19.0
1,15,106.0,0.0,0.0
2,16,4.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,4.0,0.0,0.0
5,20,23.0,0.0,0.0
6,21,7.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,70.0
1,15,Released,106.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,19.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,2.0,0.0,1.0,0.0,0.0,0.0,0.0
Released,70.0,106.0,4.0,1.0,4.0,23.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,19.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,70.0,106.0,4.0,1.0,4.0,23.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,19.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,70.0,106.0,4.0,1.0,4.0,23.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()